In [1]:
from collections import Counter
import csv
import datetime as dt
import itertools
import pandas as pd
from pandas.plotting import table
import math
import numpy as np
import regex as re
from matplotlib import pyplot as plt

# Import Datasets

In [2]:
business = pd.read_csv('yelp_academic_dataset_business.csv')
review = pd.read_csv('yelp_academic_dataset_review.csv')
checkin = pd.read_csv('yelp_academic_dataset_checkin.csv')

In [3]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [4]:
review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [5]:
checkin.head()

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


In [6]:
bus_ckn = business.merge(checkin, how = 'inner', on = 'business_id')
bus_ckn = bus_ckn.rename(columns = {'date': 'checkin_dates'})
bus_ckn = bus_ckn.dropna().reset_index(drop = True)

In [7]:
bus_ckn.shape

(104058, 15)

In [8]:
bus_ckn.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,checkin_dates
0,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...","2011-12-12 23:30:26, 2014-05-23 20:31:34, 2014..."
1,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...","2010-07-25 04:30:06, 2010-08-31 04:07:22, 2010..."
2,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...","2010-08-18 17:05:36, 2010-11-25 17:45:31, 2011..."
3,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...","2019-06-28 21:35:09, 2019-09-12 21:07:48, 2019..."
4,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...","2012-12-16 05:27:33, 2013-02-15 04:00:46, 2013..."


# Data Cleaning

## checkin_dates

In [9]:
ckn_dts = pd.Series(bus_ckn['checkin_dates']).str.split(", ")
ckn_dts1 = pd.DataFrame(ckn_dts)
ckn_dts1.head()

,checkin_dates
0,"[2011-12-12 23:30:26, 2014-05-23 20:31:34, 201..."
1,"[2010-07-25 04:30:06, 2010-08-31 04:07:22, 201..."
2,"[2010-08-18 17:05:36, 2010-11-25 17:45:31, 201..."
3,"[2019-06-28 21:35:09, 2019-09-12 21:07:48, 201..."
4,"[2012-12-16 05:27:33, 2013-02-15 04:00:46, 201..."


In [10]:
ckn_dts_mins = list(range(ckn_dts1.shape[0]))
ckn_dts_maxs = list(range(ckn_dts1.shape[0]))
num_ckn_dts = list(range(ckn_dts1.shape[0]))
ckn_dts_freqs = list(range(ckn_dts1.shape[0]))
for i in range(ckn_dts1.shape[0]):
    ckn_dts_ls = ckn_dts1.iloc[i, 0]
    ckn_dts_len = len(ckn_dts_ls)
    ckn_dts_values = list(range(ckn_dts_len))
    ckn_dts_diff = list(range(ckn_dts_len-1))
    for j in range(ckn_dts_len):
        ckn_dts_value = dt.datetime.strptime(ckn_dts_ls[j], '%Y-%m-%d %H:%M:%S')
        ckn_dts_value = ckn_dts_value.date()
        ckn_dts_values[j] = ckn_dts_value
        if ckn_dts_len > 1:
            if j >= 1:
                ckn_dts_diff[j-1] = ckn_dts_values[j] - ckn_dts_values[j-1]
            else:
                ckn_dts_diff = ckn_dts_diff
        else:
            ckn_dts_diff = [dt.timedelta(0)]
    ckn_dts_mins[i] = min(ckn_dts_values)
    ckn_dts_maxs[i] = max(ckn_dts_values)
    num_ckn_dts[i] = ckn_dts_len
    ckn_dts_freqs[i] = sum(ckn_dts_diff, dt.timedelta(0))/len(ckn_dts_diff)
bus_ckn['earliest_checkin_date'] = ckn_dts_mins
bus_ckn['latest_checkin_date'] = ckn_dts_maxs
bus_ckn['number_of_checkins'] = num_ckn_dts
bus_ckn['average_checkin_frequency'] = ckn_dts_freqs

In [11]:
bus_ckn['average_checkin_frequency'] = bus_ckn['average_checkin_frequency'].dt.days
bus_ckn = bus_ckn.drop(columns = ['checkin_dates'])
bus_ckn.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,earliest_checkin_date,latest_checkin_date,number_of_checkins,average_checkin_frequency
0,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",2011-12-12,2020-12-12,15,234
1,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...",2010-07-25,2019-05-08,401,8
2,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",2010-08-18,2021-11-27,335,12
3,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...",2019-06-28,2021-12-17,14,69
4,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...",2012-12-16,2022-01-13,22,157


In [12]:
# earliest checkin dates range from 2009-12-30 to 2022-01-18
bus_ckn['earliest_checkin_date'].sort_values()

64288     2009-12-30
11940     2009-12-31
11594     2010-01-16
17580     2010-01-16
100374    2010-01-16
             ...    
50986     2022-01-16
87386     2022-01-17
80203     2022-01-17
48375     2022-01-18
61183     2022-01-18
Name: earliest_checkin_date, Length: 104058, dtype: object

In [13]:
# latest checkin dates range from 2010-02-07 to 2022-01-19
bus_ckn['latest_checkin_date'].sort_values()

93436    2010-02-07
73390    2010-02-15
55919    2010-02-23
71235    2010-02-28
4325     2010-03-03
            ...    
30953    2022-01-19
18079    2022-01-19
25690    2022-01-19
79767    2022-01-19
93486    2022-01-19
Name: latest_checkin_date, Length: 104058, dtype: object

In [19]:
# filter out business that were not close prior to Jan 20th, 2020
bus_2020_df = bus_ckn.loc[((bus_ckn['is_open'] == 0) & (bus_ckn['latest_checkin_date'] >= dt.date(2020, 1, 20))) | (bus_ckn['is_open'] == 1)].reset_index(drop = True)
bus_2020_df.shape

(85445, 18)

In [20]:
bus_2020_df.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours', 'earliest_checkin_date',
       'latest_checkin_date', 'number_of_checkins',
       'average_checkin_frequency'],
      dtype='object')

### attributes

In [16]:
atr = bus_2020_df.iloc[:, [0, 11]]
atr.head()

,business_id,attributes
0,mpf3x-BjTdTEA3yCZrAYPw,{'BusinessAcceptsCreditCards': 'True'}
1,MTSW4McQd7CbVtyjqoe9mw,"{'RestaurantsDelivery': 'False', 'OutdoorSeati..."
2,mWMc6_wTdE0EUBKIGXDVfA,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc..."
3,CF33F8-E6oudUQ46HnavjQ,"{'BusinessParking': 'None', 'BusinessAcceptsCr..."
4,n_0UpQx1hsNbnPUSlodU8w,"{'BusinessAcceptsCreditCards': 'True', 'Restau..."


In [17]:
atr_dict = atr['attributes']
atr_dict = atr_dict.str.strip()
atr_dict = atr_dict.str.replace(r"u\'", "\'", regex = True)
atr_dict = atr_dict.str.replace(r"\"", "\'", regex = True)
atr_dict = atr_dict.str.replace(r"^\{", "", regex = True)
atr_dict = atr_dict.str.replace(r"\}$", "", regex = True)
atr_dict = atr_dict.str.split("', '")
atr_dict.head()

0               ['BusinessAcceptsCreditCards': 'True']
1    ['RestaurantsDelivery': 'False, OutdoorSeating...
2    ['BusinessAcceptsCreditCards': 'True, Wheelcha...
3    ['BusinessParking': 'None, BusinessAcceptsCred...
4    ['BusinessAcceptsCreditCards': 'True, Restaura...
Name: attributes, dtype: object

In [56]:
atr_df = pd.DataFrame(atr_dict)
atr_types = list()
for i in range(atr_df.shape[0]):
    atr_type = list()
    atr_df.iloc[i, 0] = pd.Series(atr_df.iloc[i, 0]).str.replace(r"\'", "", regex = True)
    atr_type = pd.Series(atr_df.iloc[i, 0]).str.extract(r"^(.+?):", expand = False)
    
    atr_df.iloc[i, 0] = pd.Series(([x] if isinstance(x, str) else x) for x in atr_df.iloc[i, 0])
    atr_df.iloc[i, 0] = list(itertools.chain(*atr_df.iloc[i, 0]))
    
    atr_type = pd.Series(([x] if isinstance(x, str) else x) for x in atr_type)
    atr_type = list(itertools.chain(*atr_type))
    atr_types.extend(atr_type)
atr_type1 = list(set(atr_types))
atr_type1

['CoatCheck',
 'RestaurantsAttire',
 'Smoking',
 'Caters',
 'RestaurantsTakeOut',
 'AcceptsInsurance',
 'HasTV',
 'RestaurantsDelivery',
 'RestaurantsGoodForGroups',
 'HappyHour',
 'Music',
 'Open24Hours',
 'DietaryRestrictions',
 'Alcohol',
 'BusinessAcceptsBitcoin',
 'ByAppointmentOnly',
 'RestaurantsPriceRange2',
 'BYOB',
 'GoodForKids',
 'WheelchairAccessible',
 'BikeParking',
 'RestaurantsTableService',
 'DogsAllowed',
 'HairSpecializesIn',
 'BusinessParking',
 'DriveThr',
 'Ambience',
 'AgesAllowed',
 'BusinessAcceptsCreditCards',
 'RestaurantsCounterService',
 'RestaurantsReservations',
 'OutdoorSeating',
 'BestNights',
 'GoodForMeal',
 'BYOBCorkage',
 'GoodForDancing',
 'NoiseLevel',
 'Corkage',
 'WiFi']

In [58]:
for i in range(len(atr_type1)):
    atr_type = str(atr_type1[i])
    atr_values = list(range(atr_df.shape[0]))
    for j in range(atr_df.shape[0]):
        atr_test = pd.Series(atr_df.iloc[j, 0]).str.contains(atr_type)
        if list(atr_test).count(True) == 0:
            atr_value = "NotApplicable"
        else:
            atr_index = list(atr_test).index(True)
            atr_value = pd.Series(atr_df.iloc[j, 0][atr_index]).str.extract(": (.+?)$", expand = False)[0]
            if (atr_value is None) or (atr_value == "None") or (atr_value == ""):
                atr_value = "NotApplicable"
        atr_values[j] = atr_value
    atr_df.insert(1+i, atr_type, atr_values)
atr_df.head()

ValueError: cannot insert CoatCheck, already exists

In [59]:
for i in range(len(atr_type1)):
    col_name = atr_type1[i]
    print(atr_df[col_name].value_counts())

NotApplicable    80845
False             4246
True               354
Name: CoatCheck, dtype: int64
NotApplicable    61479
casual           23489
dressy             435
formal              42
Name: RestaurantsAttire, dtype: int64
NotApplicable    81788
no                1985
outdoor           1419
yes                253
Name: Smoking, dtype: int64
NotApplicable    56923
True             15767
False            12755
Name: Caters, dtype: int64
NotApplicable    46691
True             35433
False             3321
Name: RestaurantsTakeOut, dtype: int64
NotApplicable    81072
True              3046
False             1327
Name: AcceptsInsurance, dtype: int64
NotApplicable    55898
True             23653
False             5894
Name: HasTV, dtype: int64
NotApplicable    50156
True             25452
False             9837
Name: RestaurantsDelivery, dtype: int64
NotApplicable    58164
True             23748
False             3533
Name: RestaurantsGoodForGroups, dtype: int64
NotApplicable    73674


Name: AgesAllowed, dtype: int64
True             73149
NotApplicable     9519
False             2777
Name: BusinessAcceptsCreditCards, dtype: int64
NotApplicable    85427
True                15
False                3
Name: RestaurantsCounterService, dtype: int64
NotApplicable    56592
False            19837
True              9016
Name: RestaurantsReservations, dtype: int64
NotApplicable    55744
True             14967
False            14734
Name: OutdoorSeating, dtype: int64
NotApplicable                                                                                                        80749
{monday: False, tuesday: False, friday: False, wednesday: False, thursday: False, sunday: False, saturday: False}      773
{monday: False, tuesday: False, friday: True, wednesday: False, thursday: True, sunday: False, saturday: True}         728
{monday: False, tuesday: False, friday: True, wednesday: False, thursday: False, sunday: True, saturday: True}         310
{monday: False, tuesday: Fal

In [61]:
# drop attribute columns "Open24Hours", "DietaryRestrictions", "HairSpecializesIn", "AgesAllowed", "RestaurantsCounterService", "BYOBCorkage"
atr_df = atr_df.drop(columns = ['attributes', 'Open24Hours', 'DietaryRestrictions', 'HairSpecializesIn', 'AgesAllowed', 'RestaurantsCounterService', 'BYOBCorkage'])

### GoodForMeal

In [64]:
gfm = pd.Series(atr_df['GoodForMeal']).str.strip("{|}")
gfm1 = pd.Series(gfm).str.split(", ")
gfm_df = pd.DataFrame(gfm1)
gfm_df.head(20)

,GoodForMeal
0,[NotApplicable]
1,[NotApplicable]
2,[NotApplicable]
3,[NotApplicable]
4,[NotApplicable]
5,[NotApplicable]
6,[NotApplicable]
7,"[dessert: False, latenight: False, lunch: Fals..."
8,[NotApplicable]
9,"[dessert: True, latenight: None, lunch: None, ..."


In [65]:
gfm_types = ['breakfast', 'brunch', 'lunch', 'dinner', 'latenight', 'dessert']
for i in range(gfm_df.shape[0]):
    if gfm_df.iloc[i, 0] != ["NotApplicable"]:
        gfm_values = []
        for gfm_type in gfm_types:
            gfm_test = pd.Series(gfm_df.iloc[i, 0]).str.contains(gfm_type)
            if list(gfm_test).count(True) != 0:
                gfm_index = list(gfm_test).index(True)
                gfm_value = pd.Series(gfm_df.iloc[i, 0][gfm_index]).str.extract(": (.+?)$", expand = False)[0]
                if (gfm_value == True) or (gfm_value == "True"):
                    gfm_values.append(gfm_type)
                elif (gfm_value == False) or (gfm_value == "False") or (gfm_value == "None"):
                    gfm_values = gfm_values
            else:
                gfm_values = gfm_values
        if gfm_values == []:
            gfm_df.iloc[i, 0] = "None"
        else:
            gfm_df.iloc[i, 0] = gfm_values
    else:
        gfm_df.iloc[i, 0] = "NotApplicable"
gfm_df.head(20)

,GoodForMeal
0,NotApplicable
1,NotApplicable
2,NotApplicable
3,NotApplicable
4,NotApplicable
5,NotApplicable
6,NotApplicable
7,None
8,NotApplicable
9,"[dinner, dessert]"


In [66]:
gfm_df['GoodForMeal'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


NotApplicable                              63647
None                                        6229
[lunch, dinner]                             5956
[dinner]                                    2479
[lunch]                                     2039
                                           ...  
[brunch, dinner, latenight, dessert]           2
[breakfast, brunch, latenight, dessert]        1
[brunch, lunch, latenight, dessert]            1
[brunch, latenight, dessert]                   1
[brunch, lunch, latenight]                     1
Name: GoodForMeal, Length: 62, dtype: int64

### Ambience

In [68]:
# touristy, hipster, romantic, divey, intimate, trendy, upscale, classy, casual
amb = pd.Series(atr_df['Ambience']).str.strip("{|}")
amb1 = pd.Series(amb).str.split(", ")
amb_df = pd.DataFrame(amb1)
amb_df.head(20)

,Ambience
0,[NotApplicable]
1,[NotApplicable]
2,[NotApplicable]
3,[NotApplicable]
4,[NotApplicable]
5,[NotApplicable]
6,[NotApplicable]
7,"[touristy: None, hipster: None, romantic: Fals..."
8,[NotApplicable]
9,"[touristy: False, hipster: False, romantic: Fa..."


In [69]:
amb_types = ["touristy", "hipster", "romantic", "divey", "intimate", "trendy", "upscale", "classy", "casual"]
for i in range(amb_df.shape[0]):
    if amb_df.iloc[i, 0] != ["NotApplicable"]:
        amb_values = []
        for amb_type in amb_types:
            amb_test = pd.Series(amb_df.iloc[i, 0]).str.contains(amb_type)
            if list(amb_test).count(True) != 0:
                amb_index = list(amb_test).index(True)
                amb_value = pd.Series(amb_df.iloc[i, 0][amb_index]).str.extract(": (.+?)$", expand = False)[0]
                if (amb_value == True) or (amb_value == "True"):
                    amb_values.append(amb_type)
                elif (amb_value == False) or (amb_value == "False") or (amb_value == "None"):
                    amb_values = amb_values
            else:
                amb_values = amb_values
        if amb_values == []:
            amb_df.iloc[i, 0] = "None"
        else:
            amb_df.iloc[i, 0] = amb_values
    else:
        amb_df.iloc[i, 0] = "NotApplicable"
amb_df.head(20)

,Ambience
0,NotApplicable
1,NotApplicable
2,NotApplicable
3,NotApplicable
4,NotApplicable
5,NotApplicable
6,NotApplicable
7,[casual]
8,NotApplicable
9,[classy]


### Music

In [70]:
# dj, background_music, no_music, jukebox, live, video, karaoke
mus = pd.Series(atr_df['Music']).str.strip("{|}")
mus1 = pd.Series(mus).str.split(", ")
mus_df = pd.DataFrame(mus1)
mus_df.head(20)

,Music
0,[NotApplicable]
1,[NotApplicable]
2,[NotApplicable]
3,[NotApplicable]
4,[NotApplicable]
5,[NotApplicable]
6,[NotApplicable]
7,[NotApplicable]
8,[NotApplicable]
9,[NotApplicable]


In [71]:
mus_types = ["dj", "background_music", "no_music", "jukebox", "live", "video", "karaoke"]
for i in range(mus_df.shape[0]):
    if mus_df.iloc[i, 0] != ["NotApplicable"]:
        mus_values = []
        for mus_type in mus_types:
            mus_test = pd.Series(mus_df.iloc[i, 0]).str.contains(mus_type)
            if list(mus_test).count(True) != 0:
                mus_index = list(mus_test).index(True)
                mus_value = pd.Series(mus_df.iloc[i, 0][mus_index]).str.extract(": (.+?)$", expand = False)[0]
                if (mus_value == True) or (mus_value == "True"):
                    mus_values.append(mus_type)
                elif (mus_value == False) or (mus_value == "False") or (mus_value == "None"):
                    mus_values = mus_values
            else:
                mus_values = mus_values
        if mus_values == []:
            mus_df.iloc[i, 0] = "other_music"
        elif mus_values == ["no_music"]:
            mus_df.iloc[i, 0] = "None"
        elif (len(mus_values) > 1) and ("no_music" in mus_values):
            mus_df.iloc[i, 0] = mus_values - ["no_music"]
        else:
            mus_df.iloc[i, 0] = mus_values
    else:
        mus_df.iloc[i, 0] = "NotApplicable"
mus_df.head(20)

,Music
0,NotApplicable
1,NotApplicable
2,NotApplicable
3,NotApplicable
4,NotApplicable
5,NotApplicable
6,NotApplicable
7,NotApplicable
8,NotApplicable
9,NotApplicable


### BusinessParking

In [72]:
# garage, street, validated, lot, valet
bsp = pd.Series(atr_df['BusinessParking']).str.strip("{|}")
bsp1 = pd.Series(bsp).str.split(", ")
bsp_df = pd.DataFrame(bsp1)
bsp_df.head(20)

,BusinessParking
0,[NotApplicable]
1,"[garage: False, street: True, validated: False..."
2,"[garage: None, street: None, validated: None, ..."
3,[NotApplicable]
4,"[garage: False, street: False, validated: Fals..."
5,"[garage: False, street: False, validated: Fals..."
6,"[garage: False, street: False, validated: Fals..."
7,"[garage: None, street: False, validated: None,..."
8,[NotApplicable]
9,"[valet: False, garage: None, street: True, lot..."


In [73]:
bsp_types = ["garage", "street", "validated", "lot", "valet"]
for i in range(bsp_df.shape[0]):
    if bsp_df.iloc[i, 0] != ["NotApplicable"]:
        bsp_values = []
        for bsp_type in bsp_types:
            bsp_test = pd.Series(bsp_df.iloc[i, 0]).str.contains(bsp_type)
            if list(bsp_test).count(True) != 0:
                bsp_index = list(bsp_test).index(True)
                bsp_value = pd.Series(bsp_df.iloc[i, 0][bsp_index]).str.extract(": (.+?)$", expand = False)[0]
                if (bsp_value == True) or (bsp_value == "True"):
                    bsp_values.append(bsp_type)
                elif (bsp_value == False) or (bsp_value == "False") or (bsp_value == "None"):
                    bsp_values = bsp_values
            else:
                bsp_values = bsp_values
        if bsp_values == []:
            bsp_df.iloc[i, 0] = "None"
        else:
            bsp_df.iloc[i, 0] = bsp_values
    else:
        bsp_df.iloc[i, 0] = "NotApplicable"
bsp_df.head(20)

,BusinessParking
0,NotApplicable
1,[street]
2,[lot]
3,NotApplicable
4,[lot]
5,None
6,None
7,[lot]
8,NotApplicable
9,[street]


### BestNights

In [74]:
# monday, tuesday, friday, wednesday, thursday, sunday, saturday
bnt = pd.Series(atr_df['BestNights']).str.strip("{|}")
bnt1 = pd.Series(bnt).str.split(", ")
bnt_df = pd.DataFrame(bnt1)
bnt_df.head(20)

,BestNights
0,[NotApplicable]
1,[NotApplicable]
2,[NotApplicable]
3,[NotApplicable]
4,[NotApplicable]
5,[NotApplicable]
6,[NotApplicable]
7,[NotApplicable]
8,[NotApplicable]
9,[NotApplicable]


In [75]:
bnt_types = ["garage", "street", "validated", "lot", "valet"]
for i in range(bnt_df.shape[0]):
    if bnt_df.iloc[i, 0] != ["NotApplicable"]:
        bnt_values = []
        for bnt_type in bnt_types:
            bnt_test = pd.Series(bnt_df.iloc[i, 0]).str.contains(bnt_type)
            if list(bnt_test).count(True) != 0:
                bnt_index = list(bnt_test).index(True)
                bnt_value = pd.Series(bnt_df.iloc[i, 0][bnt_index]).str.extract(": (.+?)$", expand = False)[0]
                if (bnt_value == True) or (bnt_value == "True"):
                    bnt_values.append(bnt_type)
                elif (bnt_value == False) or (bnt_value == "False") or (bnt_value == "None"):
                    bnt_values = bnt_values
            else:
                bnt_values = bnt_values
        if bnt_values == []:
            bnt_df.iloc[i, 0] = "None"
        else:
            bnt_df.iloc[i, 0] = bnt_values
    else:
        bnt_df.iloc[i, 0] = "NotApplicable"
bnt_df.head(20)

,BestNights
0,NotApplicable
1,NotApplicable
2,NotApplicable
3,NotApplicable
4,NotApplicable
5,NotApplicable
6,NotApplicable
7,NotApplicable
8,NotApplicable
9,NotApplicable


In [76]:
atr_df['GoodForMeal'] = gfm_df['GoodForMeal']
atr_df['Ambience'] = amb_df['Ambience']
atr_df['Music'] = mus_df['Music']
atr_df['BusinessParking'] = bsp_df['BusinessParking']
atr_df['BestNights'] = bnt_df['BestNights']
atr_df['business_id'] = atr['business_id']
atr_df.head()
atr_df.value_counts()

NameError: name 'gfm_df' is not defined

In [21]:
bus_final_df = bus_2020_df.drop(columns = ['attributes', 'address', 'postal_code', 'hours'])
bus_final_df.head()

,business_id,name,city,state,latitude,longitude,stars,review_count,is_open,categories,earliest_checkin_date,latest_checkin_date,number_of_checkins,average_checkin_frequency
0,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,Affton,MO,38.551126,-90.335695,3.0,15,1,"Shipping Centers, Local Services, Notaries, Ma...",2011-12-12,2020-12-12,15,234
1,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",2010-08-18,2021-11-27,335,12
2,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,Green Lane,PA,40.338183,-75.471659,4.5,13,1,"Brewpubs, Breweries, Food",2019-06-28,2021-12-17,14,69
3,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,Ashland City,TN,36.269593,-87.058943,2.0,6,1,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...",2012-12-16,2022-01-13,22,157
4,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,Brentwood,MO,38.627695,-90.340465,2.5,13,1,"Sporting Goods, Fashion, Shoe Stores, Shopping...",2010-08-30,2019-06-13,79,41


### categories

In [86]:
cat = bus_final_df.iloc[:, [0, 12]]
cat.head()

,business_id,categories
0,mpf3x-BjTdTEA3yCZrAYPw,"Shipping Centers, Local Services, Notaries, Ma..."
1,MTSW4McQd7CbVtyjqoe9mw,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
2,mWMc6_wTdE0EUBKIGXDVfA,"Brewpubs, Breweries, Food"
3,CF33F8-E6oudUQ46HnavjQ,"Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
4,n_0UpQx1hsNbnPUSlodU8w,"Sporting Goods, Fashion, Shoe Stores, Shopping..."


In [87]:
cat_dict = cat['categories']
cat_dict = cat_dict.str.strip()
cat_dict = cat_dict.str.split(", ")
cat_dict.head()

0    [Shipping Centers, Local Services, Notaries, M...
1    [Restaurants, Food, Bubble Tea, Coffee & Tea, ...
2                          [Brewpubs, Breweries, Food]
3    [Burgers, Fast Food, Sandwiches, Food, Ice Cre...
4    [Sporting Goods, Fashion, Shoe Stores, Shoppin...
Name: categories, dtype: object

In [92]:
cat_df = pd.DataFrame(cat_dict)
cat_types = list()
for i in range(cat_df.shape[0]):
    cat_type = pd.Series(([x] if isinstance(x, str) else x) for x in cat_df.iloc[i, 0])
    cat_type = list(itertools.chain(*cat_type))
    cat_types.extend(cat_type)
cat_type1 = list(set(cat_types))
cat_type1

['Pool Cleaners',
 'Tabletop Games',
 'Teacher Supplies',
 'Czech',
 'Scooter Tours',
 'Architectural Tours',
 'Farming Equipment',
 'Sandblasting',
 'Mobile Dent Repair',
 'Himalayan/Nepalese',
 'Specialty Food',
 'Sicilian',
 'Optometrists',
 'Carpenters',
 'Kickboxing',
 'Argentine',
 'Public Adjusters',
 'Music & DVDs',
 'Paddleboarding',
 'Motorcycle Dealers',
 'Chimney Sweeps',
 'Haunted Houses',
 'Personal Care Services',
 'Career Counseling',
 'Breweries',
 'Grocery',
 'Hardware Stores',
 'Discount Store',
 'Painters',
 'Sex Therapists',
 'Holiday Decorations',
 'Propane',
 'Embroidery & Crochet',
 'Solar Installation',
 'Custom Cakes',
 'Rehabilitation Center',
 'Contractors',
 'Firearm Training',
 'Sailing',
 'Shaved Snow',
 'CPR Classes',
 'Fashion',
 'Criminal Defense Law',
 'Knitting Supplies',
 'Musicians',
 'Paint & Sip',
 'Petting Zoos',
 'Oxygen Bars',
 'Indonesian',
 'International Grocery',
 'Ticket Sales',
 'Astrologers',
 'Scottish',
 'Private Investigation',
 'Pat

In [100]:
cat_count = Counter(cat_types)
cat_count.most_common()

[('Restaurants', 32998),
 ('Food', 18575),
 ('Shopping', 16083),
 ('Beauty & Spas', 9653),
 ('Nightlife', 8080),
 ('Health & Medical', 7859),
 ('Bars', 7454),
 ('Automotive', 6373),
 ('Event Planning & Services', 6069),
 ('Sandwiches', 5948),
 ('Local Services', 5644),
 ('American (Traditional)', 5578),
 ('Fast Food', 5328),
 ('Coffee & Tea', 5020),
 ('Home Services', 4977),
 ('Breakfast & Brunch', 4553),
 ('Pizza', 4527),
 ('Active Life', 4498),
 ('Burgers', 4241),
 ('Fashion', 3851),
 ('American (New)', 3753),
 ('Home & Garden', 3747),
 ('Auto Repair', 3557),
 ('Hair Salons', 3398),
 ('Arts & Entertainment', 3251),
 ('Nail Salons', 3068),
 ('Mexican', 3057),
 ('Italian', 2955),
 ('Hotels & Travel', 2930),
 ('Specialty Food', 2800),
 ('Hair Removal', 2520),
 ('Seafood', 2478),
 ('Salad', 2368),
 ('Doctors', 2303),
 ('Grocery', 2276),
 ('Chicken Wings', 2247),
 ('Fitness & Instruction', 2186),
 ('Desserts', 2110),
 ('Bakeries', 2001),
 ('Hotels', 1995),
 ('Chinese', 1950),
 ('Waxing', 

## Combine with review dataset

In [80]:
review = review.rename(columns = {'date': 'review_date'})
review['review_date'] = review['review_date'].astype('datetime64')
review['review_date'] = review['review_date'].dt.date

In [81]:
bus_review = review.merge(bus_final_df, how = 'inner', on = ['business_id'])

In [82]:
bus_review.shape

(5544679, 22)

In [84]:
review_df = bus_review.loc[bus_review['review_date'] >= dt.date(2020, 1, 20)].reset_index(drop = True)
review_df.shape

(1025744, 24)

In [85]:
review_df.head()

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,review_date,name,...,stars_y,review_count,is_open,categories,earliest_checkin_date,latest_checkin_date,number_of_checkins,average_checkin_frequency,earliest_review_date,latest_review_date
0,rjto3jNNZ4ABy5MB8i3ddA,rCvoCWEMaaN8AIYQywoGQw,XQfwVwDr-v0ZS3_CbbE5Xw,4.0,0,0,0,It took us about 45-50 minutes to be seated ev...,2020-02-02,Turning Point of North Wales,...,3.0,169,1,"Restaurants, Breakfast & Brunch, Food, Juice B...",2017-04-15,2022-01-16,177,9,NaN,NaN
1,jlrm5sNt-WCF3g-Kvp8Stw,TW54JlaMAPPrxFpqvhQa-g,XQfwVwDr-v0ZS3_CbbE5Xw,1.0,0,0,0,Horrible experience. I looked on yelp to see t...,2020-08-02,Turning Point of North Wales,...,3.0,169,1,"Restaurants, Breakfast & Brunch, Food, Juice B...",2017-04-15,2022-01-16,177,9,NaN,NaN
2,Tqwm99IUIagqHo8Fi_7ong,GqSA5t_OD1uxDZMwVLpoXA,XQfwVwDr-v0ZS3_CbbE5Xw,4.0,0,0,0,Been here multiple times for breakfast and eve...,2020-02-25,Turning Point of North Wales,...,3.0,169,1,"Restaurants, Breakfast & Brunch, Food, Juice B...",2017-04-15,2022-01-16,177,9,NaN,NaN
3,6mqdeMPwfAUour-cso3HAg,rslPC4SPqIEp06X20TUn3w,XQfwVwDr-v0ZS3_CbbE5Xw,5.0,0,0,0,I'm very surprised by the negative reviews on ...,2020-02-16,Turning Point of North Wales,...,3.0,169,1,"Restaurants, Breakfast & Brunch, Food, Juice B...",2017-04-15,2022-01-16,177,9,NaN,NaN
4,RxFzeqmEsc-lKqn5VsMnGw,OqX0IKY2ilJUQYzna0qc_A,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,This place is OKAY. Maybe I should try a diffe...,2020-11-14,Turning Point of North Wales,...,3.0,169,1,"Restaurants, Breakfast & Brunch, Food, Juice B...",2017-04-15,2022-01-16,177,9,NaN,NaN


In [28]:
bus_final_df.shape

(85445, 14)

In [29]:
list(review_df['business_id'].unique())

['XQfwVwDr-v0ZS3_CbbE5Xw',
 'kxX2SOes4o-D3ZQBkiMRfA',
 'gmjsEdUsKpj9Xxu6pdjH0g',
 'gebiRewfieSdtt17PTW6Zg',
 'uMvVYRgGNXf5boolA9HXTw',
 'EQ-TZ2eeD_E0BHuvoaeG5Q',
 'lj-E32x9_FA7GmUrBGBEWg',
 'RZtGWDLCAtuipwaZ-UfjmQ',
 'otQS34_MymijPTdNBoBdCw',
 'BVndHaLihEYbr76Z0CMEGw',
 'YtSqYv1Q_pOltsVPSx54SA',
 'rBdG_23USc7DletfZ11xGA',
 'CLEWowfkj-wKYJlQDqT1aw',
 'eFvzHawVJofxSnD7TgbZtg',
 'nRKndeZLQ3eDL10UMwS2rQ',
 'ut6fi2W2YaipNOqvi7e0jw',
 'vC2qm1y3Au5czBtbhc-DNw',
 'c-IgS6Pk6vMyax7Rbr38eA',
 'x4XdNhp0Xn8lOivzc77J-g',
 'S2Ho8yLxhKAa26pBAm6rxA',
 'MWmXGQ98KbRo3vsS5nZhMA',
 'ltBBYdNzkeKdCNPDAsxwAA',
 'bMratNjTG5ZFEA6hVyr-xQ',
 'bbEXAEFr4RYHLlZ-HFssTA',
 '0pMj5xUAecW9o1P35B0AMw',
 'X8lfGPagfLzfOsx0k08NRQ',
 'IDtLPgUrqorrpqSLdfMhZQ',
 'LfYUOclwLs_SPrNAdKMjsw',
 'PY9GRfzr4nTZeINf346QOw',
 'kq5Ghhh14r-eCxlVmlyd8w',
 'Zx7n8mdt8OzLRXVzolXNhQ',
 'A2q7d-CBM2-81tVkmS4JMw',
 'oBhJuukGRqPVvYBfTkhuZA',
 'eaJCpC6IhYphj7bwCDHTwQ',
 'W4ZEKkva9HpAdZG88juwyQ',
 'jTI5Xjk27An8ceJ6VwpXiQ',
 'j8JOZvfeHEfUWq3gEz6ABQ',
 

In [30]:
bus_df = bus_final_df.loc[bus_final_df['business_id'].isin(list(review_df['business_id'].unique()))]
bus_df.shape

(75820, 14)

In [31]:
bus_df.head()

,business_id,name,city,state,latitude,longitude,stars,review_count,is_open,categories,earliest_checkin_date,latest_checkin_date,number_of_checkins,average_checkin_frequency
0,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,Affton,MO,38.551126,-90.335695,3.0,15,1,"Shipping Centers, Local Services, Notaries, Ma...",2011-12-12,2020-12-12,15,234
1,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",2010-08-18,2021-11-27,335,12
2,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,Green Lane,PA,40.338183,-75.471659,4.5,13,1,"Brewpubs, Breweries, Food",2019-06-28,2021-12-17,14,69
3,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,Ashland City,TN,36.269593,-87.058943,2.0,6,1,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...",2012-12-16,2022-01-13,22,157
5,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,Nashville,TN,36.208102,-86.768170,1.5,10,1,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",2011-01-17,2021-06-28,21,190


In [221]:
bus_df.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'categories', 'hours', 'earliest_checkin_date', 'latest_checkin_date',
       'number_of_checkins', 'average_checkin_frequency', 'CoatCheck',
       'RestaurantsAttire', 'Smoking', 'Caters', 'RestaurantsTakeOut',
       'AcceptsInsurance', 'HasTV', 'RestaurantsDelivery',
       'RestaurantsGoodForGroups', 'HappyHour', 'Music', 'Alcohol',
       'BusinessAcceptsBitcoin', 'ByAppointmentOnly', 'RestaurantsPriceRange2',
       'BYOB', 'GoodForKids', 'WheelchairAccessible', 'BikeParking',
       'RestaurantsTableService', 'DogsAllowed', 'BusinessParking', 'DriveThr',
       'Ambience', 'BusinessAcceptsCreditCards', 'RestaurantsReservations',
       'OutdoorSeating', 'BestNights', 'GoodForMeal', 'GoodForDancing',
       'NoiseLevel', 'Corkage', 'WiFi'],
      dtype='object')

In [35]:
review_df = review_df.drop(columns = ['stars_y'])
review_df = review_df.rename(columns = {'stars_x': 'stars'})

In [36]:
review_df['sentiment'] = np.where(review_df['stars']>=3.5, 1, -1)

In [37]:
review_df['influence'] = review_df.apply(lambda row: (row['useful'] + row['funny'] + row['cool'])*row['sentiment'], axis = 1)

In [41]:
review_df['influence'].value_counts().head(10)

 0    590085
 1    109724
-1     67870
 2     63960
 3     41757
-2     30781
 4     21324
-3     16884
 5     13002
-4      8879
Name: influence, dtype: int64

In [53]:
x = review_df.groupby(['business_id'])['influence'].sum()
x1 = pd.DataFrame()
x1['business_id'] = x.index
x1['influence'] = x.values
bus_df = bus_df.merge(x1, how = 'left', on = ['business_id'])
bus_df.head()

,business_id,name,city,state,latitude,longitude,stars,review_count,is_open,categories,earliest_checkin_date,latest_checkin_date,number_of_checkins,average_checkin_frequency,influence
0,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,Affton,MO,38.551126,-90.335695,3.0,15,1,"Shipping Centers, Local Services, Notaries, Ma...",2011-12-12,2020-12-12,15,234,3
1,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",2010-08-18,2021-11-27,335,12,17
2,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,Green Lane,PA,40.338183,-75.471659,4.5,13,1,"Brewpubs, Breweries, Food",2019-06-28,2021-12-17,14,69,1
3,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,Ashland City,TN,36.269593,-87.058943,2.0,6,1,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...",2012-12-16,2022-01-13,22,157,0
4,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,Nashville,TN,36.208102,-86.768170,1.5,10,1,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",2011-01-17,2021-06-28,21,190,0


In [54]:
bus_df = bus_df.astype({'is_open': 'category', 'earliest_checkin_date': 'datetime64', 'latest_checkin_date': 'datetime64'})

In [55]:
review_df = review_df.astype({'is_open': 'category', 'earliest_checkin_date': 'datetime64', 'latest_checkin_date': 'datetime64'})

In [56]:
bus_df['city'] = bus_df['city'].replace(['Saint Louis', 'East St.Louis', 'Saint Louis,', 'St Louis'], 'St. Louis')
bus_df['city'] = bus_df['city'].replace(['St Petersburg', 'Saint Petersburg'], 'St. Petersburg')
bus_df['city'] = bus_df['city'].replace(['Saint Albert', 'St Albert'], 'St. Albert')
bus_df['city'] = bus_df['city'].replace(['Saint Rose', 'St Rose'], 'St. Rose')
bus_df['city'] = bus_df['city'].replace(['TRENTON', 'NASHVILLE', 'NEW ORLEANS', 'Boise City', 'Pinellas Park', 'King Of Prussia', 'East Norristown', 'UPPER MORELAND', 'RADNOR', 'Goodlettsville', 'Springhill', 'Washington Twp', 'Fairview Hts', 'Saint Peters', 'St Pete Beach', 'Land O\' Lakes'], ['Trenton', 'Nashville', 'New Orleans', 'Boise', 'Pinellas', 'King of Prussia', 'Norristown', 'Upper Moreland', 'Radnor', 'Goodletsville', 'Spring Hill', 'Washington Township', 'Fairview Heights', 'St. Peters', 'St. Pete Beach', 'Land O Lakes'])
bus_df['city'] = bus_df['city'].replace(['Philadelphia \(Northeast Philly\)', 'PHILADELPHIA', 'philadelphia'], 'Philadelphia')
bus_df['city'] = bus_df['city'].replace(['Tampa Bay', 'South Tampa', 'TAMPA', 'New Tampa', 'tampa', 'Tampa Palms', 'Tampa,'], 'Tampa')
bus_df['city'] = bus_df['city'].replace(['TUCSON', 'Corona de Tucson', 'Corona De Tucson', 'South Tucson', 'tucson'], 'Tucson')
bus_df['city'] = bus_df['city'].replace(['Clearwater Beach', 'CLEARWATER', 'Clearwater/ Countryside'], 'Clearwater')
bus_df['city'] = bus_df['city'].replace(['Franklinville', 'Franklin '], 'Franklin')
bus_df['city'] = bus_df['city'].replace(['Wilmington ', 'wilmington'], 'Wilmington')
bus_df['city'] = bus_df['city'].replace(['\u200bLargo', 'LARGO'], 'Largo')
bus_df['city'] = bus_df['city'].replace(['lutz', 'Lutz fl'], 'Lutz')
bus_df['city'] = bus_df['city'].replace(['RIVERVIEW', 'Riverview Fl', 'River View'], 'Riverview')
bus_df['city'] = bus_df['city'].replace(['Eagleville', 'Eagle,'], 'Eagle')
bus_df['city'] = bus_df['city'].replace(['Indianapolis city \(balance\)', 'indianapolis'], 'Indianapolis')
bus_df['city'] = bus_df['city'].replace(['O\'fallon', 'O\'Fallon'], 'O Fallon')
bus_df['city'] = bus_df['city'].replace(['West Deptford', 'Deptford Twp', 'Deptford Township'], 'Deptford')
bus_df['city'] = bus_df['city'].replace(['Saint Charles', 'St Charles'], 'St. Charles')
bus_df['city'] = bus_df['city'].replace(['Mt.Juliet', 'Mount Juliet', 'Mt Juliet'], 'Mt. Juliet')
bus_df['city'] = bus_df['city'].replace(['Mt.Laurel', 'Mount Laurel', 'Mt Laurel'], 'Mt. Laurel')
bus_df['city'] = bus_df['city'].replace(['Mt.Holly', 'Mount Holly', 'Mt Holly'], 'Mt. Holly')

In [57]:
review_df['city'] = review_df['city'].replace(['Saint Louis', 'East St.Louis', 'Saint Louis,', 'St Louis'], 'St. Louis')
review_df['city'] = review_df['city'].replace(['St Petersburg', 'Saint Petersburg'], 'St. Petersburg')
review_df['city'] = review_df['city'].replace(['Saint Albert', 'St Albert'], 'St. Albert')
review_df['city'] = review_df['city'].replace(['Saint Rose', 'St Rose'], 'St. Rose')
review_df['city'] = review_df['city'].replace(['TRENTON', 'NASHVILLE', 'NEW ORLEANS', 'Boise City', 'Pinellas Park', 'King Of Prussia', 'East Norristown', 'UPPER MORELAND', 'RADNOR', 'Goodlettsville', 'Springhill', 'Washington Twp', 'Fairview Hts', 'Saint Peters', 'St Pete Beach', 'Land O\' Lakes', 'BRANDON', 'wimauma'], ['Trenton', 'Nashville', 'New Orleans', 'Boise', 'Pinellas', 'King of Prussia', 'Norristown', 'Upper Moreland', 'Radnor', 'Goodletsville', 'Spring Hill', 'Washington Township', 'Fairview Heights', 'St. Peters', 'St. Pete Beach', 'Land O Lakes', 'Brandon', 'Wimauma'])
review_df['city'] = review_df['city'].replace(['Philadelphia \(Northeast Philly\)', 'PHILADELPHIA', 'philadelphia'], 'Philadelphia')
review_df['city'] = review_df['city'].replace(['Tampa Bay', 'South Tampa', 'TAMPA', 'New Tampa', 'tampa', 'Tampa Palms', 'Tampa,'], 'Tampa')
review_df['city'] = review_df['city'].replace(['TUCSON', 'Corona de Tucson', 'Corona De Tucson', 'South Tucson', 'tucson'], 'Tucson')
review_df['city'] = review_df['city'].replace(['Clearwater Beach', 'CLEARWATER', 'Clearwater/ Countryside'], 'Clearwater')
review_df['city'] = review_df['city'].replace(['Franklinville', 'Franklin '], 'Franklin')
review_df['city'] = review_df['city'].replace(['Wilmington ', 'wilmington'], 'Wilmington')
review_df['city'] = review_df['city'].replace(['\u200bLargo', 'LARGO'], 'Largo')
review_df['city'] = review_df['city'].replace(['lutz', 'Lutz fl'], 'Lutz')
review_df['city'] = review_df['city'].replace(['RIVERVIEW', 'Riverview Fl', 'River View'], 'Riverview')
review_df['city'] = review_df['city'].replace(['Eagleville', 'Eagle,'], 'Eagle')
review_df['city'] = review_df['city'].replace(['Indianapolis city \(balance\)', 'indianapolis'], 'Indianapolis')
review_df['city'] = review_df['city'].replace(['O\'fallon', 'O\'Fallon'], 'O Fallon')
review_df['city'] = review_df['city'].replace(['West Deptford', 'Deptford Twp', 'Deptford Township'], 'Deptford')
review_df['city'] = review_df['city'].replace(['Saint Charles', 'St Charles'], 'St. Charles')
review_df['city'] = review_df['city'].replace(['Mt.Juliet', 'Mount Juliet', 'Mt Juliet'], 'Mt. Juliet')
review_df['city'] = review_df['city'].replace(['Mt.Laurel', 'Mount Laurel', 'Mt Laurel'], 'Mt. Laurel')
review_df['city'] = review_df['city'].replace(['Mt.Holly', 'Mount Holly', 'Mt Holly'], 'Mt. Holly')

In [74]:
bus_df['name'].drop_duplicates(keep = False, inplace = True)
chain_bus = pd.DataFrame()
chain_bus['name'] = bus_df['name'].value_counts().index
chain_bus['count'] = bus_df['name'].value_counts().values
chain_bus1 = list(chain_bus.loc[chain_bus['count']>1]['name'])
bus_df['chain'] = np.where(bus_df['name'].isin(chain_bus1), 1, 0)

In [77]:
review_df['stars'].value_counts()

5.0    552303
1.0    187171
4.0    142625
3.0     74037
2.0     69608
Name: stars, dtype: int64

In [79]:
review_df.groupby(['business_id'])['review_date']
review_df

,review_id,user_id,business_id,stars,useful,funny,cool,text,review_date,name,...,longitude,review_count,is_open,categories,earliest_checkin_date,latest_checkin_date,number_of_checkins,average_checkin_frequency,sentiment,influence
0,rjto3jNNZ4ABy5MB8i3ddA,rCvoCWEMaaN8AIYQywoGQw,XQfwVwDr-v0ZS3_CbbE5Xw,4.0,0,0,0,It took us about 45-50 minutes to be seated ev...,2020-02-02,Turning Point of North Wales,...,-75.223639,169,1,"Restaurants, Breakfast & Brunch, Food, Juice B...",2017-04-15,2022-01-16,177,9,1,0
1,jlrm5sNt-WCF3g-Kvp8Stw,TW54JlaMAPPrxFpqvhQa-g,XQfwVwDr-v0ZS3_CbbE5Xw,1.0,0,0,0,Horrible experience. I looked on yelp to see t...,2020-08-02,Turning Point of North Wales,...,-75.223639,169,1,"Restaurants, Breakfast & Brunch, Food, Juice B...",2017-04-15,2022-01-16,177,9,-1,0
2,Tqwm99IUIagqHo8Fi_7ong,GqSA5t_OD1uxDZMwVLpoXA,XQfwVwDr-v0ZS3_CbbE5Xw,4.0,0,0,0,Been here multiple times for breakfast and eve...,2020-02-25,Turning Point of North Wales,...,-75.223639,169,1,"Restaurants, Breakfast & Brunch, Food, Juice B...",2017-04-15,2022-01-16,177,9,1,0
3,6mqdeMPwfAUour-cso3HAg,rslPC4SPqIEp06X20TUn3w,XQfwVwDr-v0ZS3_CbbE5Xw,5.0,0,0,0,I'm very surprised by the negative reviews on ...,2020-02-16,Turning Point of North Wales,...,-75.223639,169,1,"Restaurants, Breakfast & Brunch, Food, Juice B...",2017-04-15,2022-01-16,177,9,1,0
4,RxFzeqmEsc-lKqn5VsMnGw,OqX0IKY2ilJUQYzna0qc_A,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,This place is OKAY. Maybe I should try a diffe...,2020-11-14,Turning Point of North Wales,...,-75.223639,169,1,"Restaurants, Breakfast & Brunch, Food, Juice B...",2017-04-15,2022-01-16,177,9,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025739,nkhDNmwRvOafKVVJjS7ADg,nnf6O3EBML84OzwLJKh-DQ,nRh8d_hu5qUPP4MBzMS0sw,1.0,9,10,0,Not a single sales person was wearing a mask a...,2020-09-25,Consign Furniture Boise,...,-116.405328,8,1,"Home & Garden, Furniture Stores, Mattresses, F...",2019-08-30,2021-09-02,7,122,-1,-19
1025740,Acf5zcbZLOuN32dS24IObw,Ik4CAdGjhx9YABc5tIW-yg,nRh8d_hu5qUPP4MBzMS0sw,5.0,3,0,0,LOTS of furniture to choose from. Staff was re...,2020-02-06,Consign Furniture Boise,...,-116.405328,8,1,"Home & Garden, Furniture Stores, Mattresses, F...",2019-08-30,2021-09-02,7,122,1,3
1025741,7MZu6bFdFoqr7MyPKqvoew,aYveEctPYcZiubXyEgLhTA,vI4vyi1dfG93oAiSRFDymA,5.0,4,1,2,Love this office. All the staff us super kind ...,2020-02-17,Aesthetic Dermatology Associates,...,-75.501140,5,1,"Medical Spas, Doctors, Laser Hair Removal, Der...",2020-02-12,2020-02-12,1,0,1,7
1025742,3emHuLQvuAw0l66TzSWmKA,oz-So7Kwo5tW51HrT-BgIg,vI4vyi1dfG93oAiSRFDymA,1.0,0,0,0,It was a HUGE mistake to give this place a try...,2021-09-22,Aesthetic Dermatology Associates,...,-75.501140,5,1,"Medical Spas, Doctors, Laser Hair Removal, Der...",2020-02-12,2020-02-12,1,0,-1,0


In [75]:
bus_df.to_csv("bus_df.csv", index = False)
review_df.to_csv("review_df.csv", index = False)

KeyboardInterrupt: 